In [1]:
import gzip
from collections import defaultdict
import numpy as np
import random
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    
print "Reading data..."
data = list(readGz('train.json.gz'))
print "Done"

Reading data...
Done


In [2]:
train = data[:100000]
valid = data[100000:]

In [20]:
beta_user = defaultdict(float)
beta_item = defaultdict(float)
alp = 0
lamb = 1
userrating = defaultdict(lambda: defaultdict(int)) 
businessrating = defaultdict(lambda: defaultdict(int)) 
for d in train:
    uu,bb = d['reviewerID'],d['itemID']
    userrating[uu][bb] = d['rating']
    businessrating[bb][uu] = d['rating']
    beta_user[uu] = 0
    beta_item[bb] = 0

In [21]:
def fun8(lamb):
    times = 50
    for i in range(times):
        alp = 0
        for uu in userrating:
            for bb in userrating[uu]:
                alp += userrating[uu][bb] - (beta_user[uu] + beta_item[bb])
        alp /= len(train)
        for uu in userrating: 
            beta_user[uu] = 0
            for bb in userrating[uu]:
                beta_user[uu] += userrating[uu][bb] - (alp + beta_item[bb])
            beta_user[uu] /= (lamb + len(userrating[uu]))
        for bb in businessrating:
            beta_item[bb] = 0
            for uu in businessrating[bb]:
                beta_item[bb] += businessrating[bb][uu] - (alp + beta_user[uu])
            beta_item[bb] /= (lamb + len(businessrating[bb]))
        mse = 0
        for uu in userrating:
            for bb in userrating[uu]:
                mse += (alp + beta_user[uu] + beta_item[bb] - userrating[uu][bb]) **2
        mse /= len(train)

    validation_user = defaultdict(lambda: defaultdict(int)) 
    validation_business = defaultdict(lambda: defaultdict(int)) 
    for d in valid:
        uu, bb = d['reviewerID'], d['itemID']
        validation_user[uu][bb] = d['rating']
        validation_business[bb][uu] = d['rating']
    mse = 0
    for uu in validation_user:
        for bb in validation_user[uu]:
            mse += ((alp + (beta_user[uu] if uu in beta_user else 0) + 
                   (beta_item[bb] if bb in beta_item else 0) - 
                    validation_user[uu][bb]) **2)

    mse /= len(valid)
    return mse

In [22]:
fun8(10.0)

1.1416030979547096

In [109]:
g_u = defaultdict(float)
g_i = defaultdict(float)
for d in train:
    uu,bb = d['reviewerID'],d['itemID']
    g_u[uu] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)
    g_i[bb] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)

In [110]:
def fun88(lamb):
    times = 50
    for i in range(times):
        alp = 0
        for uu in userrating:
            for bb in userrating[uu]:
                alp += userrating[uu][bb] - (beta_user[uu] + beta_item[bb] + g_u[uu]*g_i[bb])
        alp /= len(train)
        for uu in userrating: 
            beta_user[uu] = 0
            for bb in userrating[uu]:
                beta_user[uu] += userrating[uu][bb] - (alp + beta_item[bb] + g_u[uu]*g_i[bb])
            beta_user[uu] /= (lamb + len(userrating[uu]))
        for bb in businessrating:
            beta_item[bb] = 0
            for uu in businessrating[bb]:
                beta_item[bb] += businessrating[bb][uu] - (alp + beta_user[uu] + g_u[uu]*g_i[bb])
            beta_item[bb] /= (lamb + len(businessrating[bb]))
        for uu in userrating:
            g_u[uu] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)
            for bb in userrating[uu]:
                g_u[uu] += g_i[bb] * (userrating[uu][bb] - (alp + beta_user[uu] + beta_item[bb] + g_u[uu]*g_i[bb]))
            g_u[uu] /= lamb
        for bb in businessrating:
            g_i[bb] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)
            for uu in businessrating[bb]:
                g_i[bb] += g_u[uu] * (businessrating[bb][uu] - (alp + beta_user[uu] + beta_item[bb] + g_u[uu]*g_i[bb]))
            g_i[bb] /= lamb
        mse = 0
        for uu in userrating:
            for bb in userrating[uu]:
                mse += (alp + beta_user[uu] + beta_item[bb] - userrating[uu][bb]) **2
        mse /= len(train)

    validation_user = defaultdict(lambda: defaultdict(int)) 
    validation_business = defaultdict(lambda: defaultdict(int)) 
    for d in valid:
        uu, bb = d['reviewerID'], d['itemID']
        validation_user[uu][bb] = d['rating']
        validation_business[bb][uu] = d['rating']
    mse = 0
    for uu in validation_user:
        for bb in validation_user[uu]:
            mse += ((alp + (beta_user[uu] if uu in beta_user else 0) + 
                   (beta_item[bb] if bb in beta_item else 0) + 
                     (g_u[uu] if uu in g_u else 0) * (g_i[bb] if bb in g_i else 0) - 
                    validation_user[uu][bb]) **2)

    mse /= len(valid)
    return mse, alp, beta_user, beta_item, g_u, g_i

In [25]:
fun88(10.0)

1.1416230899180426

In [10]:
# find the best lamb
lambdas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
mse = []
for l in lambdas:
    mse.append(fun88(l))
for i in range(len(mse)):
    m = mse[i]
    l = lambdas[i]
    print str(m) +'\t'+ str(l)

nan	0.0001
nan	0.001
nan	0.01
nan	0.1
nan	1.0
nan	10.0
nan	100.0
nan	1000.0
nan	10000.0


In [27]:
fun88(10.0)

nan

In [111]:
train = data[:199999]
valid = data[199999:]

In [112]:
def init():
    
    alp = 0
    lamb = 1
     
    for d in train:
        uu,bb = d['reviewerID'],d['itemID']
        userrating[uu][bb] = d['rating']
        businessrating[bb][uu] = d['rating']
        beta_user[uu] = 0
        beta_item[bb] = 0
    
    for d in train:
        uu,bb = d['reviewerID'],d['itemID']
        g_u[uu] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)
        g_i[bb] = np.random.normal(0.0,.1)#random.uniform(-.5,.5)

In [113]:
m = []
alpha = []
bu = []
bi = []
gu = []
gi = []
beta_user = defaultdict(float)
beta_item = defaultdict(float)
userrating = defaultdict(lambda: defaultdict(int)) 
businessrating = defaultdict(lambda: defaultdict(int))
g_u = defaultdict(float)
g_i = defaultdict(float)

In [114]:
def rating(lam1, lam2):
    
    init()
    mse1 = fun8(lam1)
    print "mse1 : " + str(mse1)
    mse2, a, b_u, b_i, g_u, g_i =  fun88(lam2)
    m.append(mse2)
    alpha.append(a)
    bu.append(b_u)
    bi.append(b_i)
    gu.append(g_u)
    gi.append(g_i)
    print "mse2 : " + str(mse2)

In [36]:
rating(10.0, 10.0)

mse1 : 1.14160309795
mse2 : 1.14162814744


In [41]:
rating(10.0, 1.0)

mse1 : 1.14160309795
mse2 : nan


In [45]:
rating(10.0, 10.0)

mse1 : 1.14160309795
mse2 : 1.14159221695


In [47]:
rating(10.0, 100.0)

mse1 : 1.14160309795
mse2 : 1.19982484824


In [48]:
rating(10.0, 9.0)

mse1 : 1.14160309795
mse2 : 1.13995760253


In [49]:
rating(10.0, 8.0)

mse1 : 1.14160309795
mse2 : 1.13861857393


In [54]:
rating(10.0, 7.0)

mse1 : 1.14160309795
mse2 : 1.13777768623


In [55]:
rating(10.0, 6.0)

mse1 : 1.14160309795
mse2 : 1.13817781729


In [56]:
rating(10.0, 5.0)

mse1 : 1.14160309795
mse2 : 1.14048701303


In [57]:
rating(10.0, 6.5)

mse1 : 1.14160309795
mse2 : 1.13768969262


In [58]:
rating(10.0, 6.75)

mse1 : 1.14160309795
mse2 : 1.13774786557


In [60]:
len(m)

11

In [94]:
rating(10.0, 10.0)

mse1 : 7.65491963125
mse2 : 7.66633016929


In [98]:
rating(10.0, 7.0)

mse1 : 7.65491963125
mse2 : 6.952621545


In [100]:
rating(10.0, 6.0)

mse1 : 7.65491963125
mse2 : 6.65580375368


In [102]:
rating(10.0, 5.0)

mse1 : 7.65491963125
mse2 : 6.3008433228


In [104]:
rating(10.0, 6.25)

mse1 : 7.65491963125
mse2 : 6.7389049974


In [106]:
rating(10.0, 6.75)

mse1 : 7.65491963125
mse2 : 6.8791423796


In [116]:
rating(10.0, 6.4)

mse1 : 7.65491963125
mse2 : 6.78258937498


In [118]:
rating(10.0, 6.5)

mse1 : 7.65491963125
mse2 : 6.81136289177


In [121]:
predictions = open("/Users/nan/Desktop/assign1_pairs_Rating.txt", 'w') 
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"): 
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-') 
    x = alpha[len(alpha)-1]
    if u in bu[len(bu)-1]:
        x += bu[len(bu)-1][u] 
    if i in bi[len(bi)-1]:
        x += bi[len(bi)-1][i]
    if u in gu[len(gu)-1]:
        if i in gi[len(gi)-1]:
            x += gu[len(gu)-1][u]*gi[len(gi)-1][i]
    predictions.write(u + '-' + i + ',' + str(x) + '\n')

predictions.close()

In [120]:
alpha

[4.228449760467453, 4.228529097541549]